# Local Sequence Alignment

This is my first computational biology project. I was inspired to undertake this project after following the MIT open course called [Foundations of Computational and Systems Biology](https://ocw.mit.edu/courses/biology/7-91j-foundations-of-computational-and-systems-biology-spring-2014/index.htm) by **Prof. Christopher Burge**. Much of my code was inspired by **Prof. Ben Langmead**, which can be found [here](https://github.com/BenLangmead/comp-genomics-class/blob/master/notebooks/CG_DP_Local.ipynb), and the Stackoverflow community. Here, I use the same sequences and scoring penalties as Prof. Langmead so i can cross-reference my output with his for correctness.

## What is it?
Local sequence alignment takes two sequences and finds two sub sequences with the best alignment based on matching bases between the two sequences. The best alignment can have base matches (eg. G to G), missmatches (eg. G to A), or gaps (eg. G to -). A common strategy to determine local sequence alignment utilizes the Smith Waterman algorithm, which is described here [here](https://en.wikipedia.org/wiki/Smith–Waterman_algorithm#Gap_penalty).

### Dynamic programming algorithm

I start by adding a couple of DNA sequences (sx and sy) for testing the algorithm, defining two arrays (one to keep track of the score (M) and another to keep track of the direction the score came from (P)), and creating a *SmithWaterman* function that fills both M and P arrays. This algoritm utilizes a linear gap penalty.

In [1]:
import numpy
        
sx = "GGTATGCTGGCGCTA"
sy = "TATATGCGGCGTTT"
M = numpy.zeros((len(sx) + 1, len(sy) + 1), dtype = int)
P = numpy.zeros((len(sx) + 1, len(sy) + 1), dtype = int)
match, indelx, indely = 1, 2, 3

def SmithWaterman(X, Y):
    match_s, mis_s, gap = 2, -4, -6
    for n in range(1, len(X) + 1):
        for m in range(1, len(Y) + 1):
            align = (M[n-1, m-1] + (match_s if X[n-1] == Y[m-1] else mis_s), match)
            indelX = (M[n, m-1] + gap, indelx)
            indelY = (M[n-1, m] + gap, indely)
            M[n, m], P[n, m] = max(align, indelX, indelY, (0, 0))
    return M

In [2]:
print(SmithWaterman(sx, sy))
print("The largest score is %s in cell %s" % (numpy.max(M), numpy.unravel_index(M.argmax(), M.shape)))

[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  2  0  2  2  0  2  0  0  0]
 [ 0  0  0  0  0  0  2  0  2  4  0  2  0  0  0]
 [ 0  2  0  2  0  2  0  0  0  0  0  0  4  2  2]
 [ 0  0  4  0  4  0  0  0  0  0  0  0  0  0  0]
 [ 0  2  0  6  0  6  0  0  0  0  0  0  2  2  2]
 [ 0  0  0  0  2  0  8  2  2  2  0  2  0  0  0]
 [ 0  0  0  0  0  0  2 10  4  0  4  0  0  0  0]
 [ 0  2  0  2  0  2  0  4  6  0  0  0  2  2  2]
 [ 0  0  0  0  0  0  4  0  6  8  2  2  0  0  0]
 [ 0  0  0  0  0  0  2  0  2  8  4  4  0  0  0]
 [ 0  0  0  0  0  0  0  4  0  2 10  4  0  0  0]
 [ 0  0  0  0  0  0  2  0  6  2  4 12  6  0  0]
 [ 0  0  0  0  0  0  0  4  0  2  4  6  8  2  0]
 [ 0  2  0  2  0  2  0  0  0  0  0  0  8 10  4]
 [ 0  0  4  0  4  0  0  0  0  0  0  0  2  4  6]]
The largest score is 12 in cell (12, 11)


### Traceback the alignment

Next I define traceback that uses the P array to determine the best alignment of sx and sy strands.

In [3]:
def traceback(m, p, X, Y):
    seqx, mid, seqy = [], [], []
    n, m = numpy.unravel_index(M.argmax(), M.shape)
    while ((n or m) > 0) and p[n, m] != 0:
        if p[n, m] == match:
            seqx.append(X[n-1])
            mid.append('|')
            seqy.append(Y[m-1])
            n -= 1
            m -= 1
        if p[n, m] == indelx:
            seqx.append('-')
            mid.append(' ')
            seqy.append(Y[m-1])
            m -= 1
        if p[n, m] == indely:
            seqx.append(X[n-1])
            mid.append(' ')
            seqy.append('-')
            n -= 1
    print("best alignment starts in cell %s \nand ends in cell %s" \
          % ((n+1, m+1), numpy.unravel_index(M.argmax(), M.shape)))
    alignment = '\n'.join(map(lambda x: ''.join(x), [seqx[::-1], mid[::-1], seqy[::-1]]))
    return alignment

In [4]:
print(traceback(M, P, sx, sy))

best alignment starts in cell (3, 3) 
and ends in cell (12, 11)
TATGCTGGCG
||||| ||||
TATGC-GGCG
